In [1]:
from sklearn.model_selection import train_test_split

from data_preprocessing import DataPreprocessor
from model_classifier import CrossSellingModel

[2024-07-14 09:59:12,361 : DEBUG : Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.]
[2024-07-14 09:59:21,257 : DEBUG : Creating converter from 7 to 5]
[2024-07-14 09:59:21,262 : DEBUG : Creating converter from 5 to 7]
[2024-07-14 09:59:21,262 : DEBUG : Creating converter from 7 to 5]
[2024-07-14 09:59:21,267 : DEBUG : Creating converter from 5 to 7]


In [2]:
train_data = DataPreprocessor('data/train.zip')

In [ ]:
train = train_data.preprocess_data()
train.head()

In [4]:
X = train.drop(columns='Response', axis=1)
y = train['Response']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"X_train.shape: {X_train.shape}")
print(f"y_train.shape: {y_train.shape}")

X_train.shape: (9203838, 12)
y_train.shape: (9203838,)


In [5]:
classifier = CrossSellingModel(X_train.shape[1])

In [6]:
classifier.train_model(X_train, y_train, epochs=100, batch_size=128, validation_split=0.2)

Epoch 1/100
57524/57524 ━━━━━━━━━━━━━━━━━━━━ 93s 2ms/step - accuracy: 0.8080 - loss: 14.7612 - val_accuracy: 0.8770 - val_loss: 0.3254
Epoch 2/100
57524/57524 ━━━━━━━━━━━━━━━━━━━━ 123s 2ms/step - accuracy: 0.8686 - loss: 0.3263 - val_accuracy: 0.8770 - val_loss: 0.3646
Epoch 3/100
57524/57524 ━━━━━━━━━━━━━━━━━━━━ 135s 2ms/step - accuracy: 0.8768 - loss: 0.3556 - val_accuracy: 0.8770 - val_loss: 0.2791
Epoch 4/100
57524/57524 ━━━━━━━━━━━━━━━━━━━━ 133s 2ms/step - accuracy: 0.8772 - loss: 0.2815 - val_accuracy: 0.8770 - val_loss: 0.2921
Epoch 5/100
57524/57524 ━━━━━━━━━━━━━━━━━━━━ 157s 2ms/step - accuracy: 0.8770 - loss: 0.2795 - val_accuracy: 0.8770 - val_loss: 0.2750
Epoch 6/100
57524/57524 ━━━━━━━━━━━━━━━━━━━━ 143s 2ms/step - accuracy: 0.8770 - loss: 0.2789 - val_accuracy: 0.8770 - val_loss: 0.2754
Epoch 7/100
57524/57524 ━━━━━━━━━━━━━━━━━━━━ 110s 2ms/step - accuracy: 0.8770 - loss: 0.2783 - val_accuracy: 0.8770 - val_loss: 0.2836
Epoch 8/100
57524/57524 ━━━━━━━━━━━━━━━━━━━━ 192s 3ms/s

In [7]:
test_data = DataPreprocessor('data/test.zip')

In [9]:
import pandas as pd

ids = test_data.df['id']
test_df = test_data.preprocess_data()
test_df.head()

C:\Users\johnp\PycharmProjects\ai_ml_projects\insurance_cross_selling\data_preprocessing.py:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['Gender'] = self.df['Gender'].replace({'Male': 0, 'Female': 1})
C:\Users\johnp\PycharmProjects\ai_ml_projects\insurance_cross_selling\data_preprocessing.py:28: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df = self.df.replace({col: {True: 1, False: 0} for col in self.df.columns})


,Gender,Age,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Vehicle_Age_1-2 Year,Vehicle_Age_< 1 Year,Vehicle_Age_> 2 Years,Vehicle_Damage_No,Vehicle_Damage_Yes
0,1,20,47.0,0,2630.0,160.0,228,0,1,0,1,0
1,0,47,28.0,0,37483.0,124.0,123,1,0,0,0,1
2,0,47,43.0,0,2630.0,26.0,271,1,0,0,0,1
3,1,22,47.0,1,24502.0,152.0,115,0,1,0,1,0
4,0,51,19.0,0,34115.0,124.0,148,1,0,0,1,0


In [11]:
predictions = classifier.get_predictions(test_df, 0.5)
submission = pd.DataFrame({'id': ids, 'Response': predictions})
submission.to_csv('submission.csv', index=False)

239684/239684 ━━━━━━━━━━━━━━━━━━━━ 197s 822us/step


In [13]:
classifier.save_model('cross_selling_model.keras')